# Hybridization of Traditional and GAN-based Augmentations
- In this kernel, we will be performing the Hybridization of **Traditional Augmentation** and **GAN-Based Augmentation** approaches. 
- In the traditional and GAN-based augmentation kernels, we have tried 3 different approaches in each. In this manner, we can have 9 different combinations for this hybridization. However, in this kernel, we will only be using a single combination. From both the kernels, we will be selecting the approach with the best performance scores, and combining them only. 
- We have used 3 performance metrics in each of the 3 different approaches, **Multi-Class Log-Loss**, **Weighted F1-Score** and **Accuracy**. We selected the approach which outperformed the other 2, in at least 2 out of these 3 different performance metrics. 
- From the traditional augmentation kernel, we will be selecting 'Augmentation based on class-wise performance' (**77.39% accuracy**), and from the GAN-based augmentation kernel too, we will be selecting 'Augmentation based on class-wise performance' (**76.64% accuracy**).
- We will apply the aforementioned approaches individually on the training dataset, and then will be merging both of the augmented datasets with the training set.

### Reference Kernels
- [Traditional Augmentation](https://www.kaggle.com/code/elemento/rw-tradaug)
- [GANs Augmentation 1](https://www.kaggle.com/code/elemento/rw-ganaug-1) and [GANs Augmentation 2](https://www.kaggle.com/code/elemento/rw-ganaug-2)

# 1. Importing the Packages & Boilerplate Code

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from shutil import copyfile
from tabulate import tabulate
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, f1_score

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import tensorflow.keras.layers as tfl

In [2]:
# Setting the seeds
SEED = 0
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Making sure that Tensorflow is able to detect the GPU
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 2. Importing the Train and Test Sets

In [4]:
# Importing the Labelled Training Dataset
print("For Train Dataset:")
df_train = pd.read_csv("../input/cifar10/train_lab_x.csv")
y_train = pd.read_csv("../input/cifar10/train_lab_y.csv")
df_train = np.array(df_train)
y_train = np.array(y_train)
print(df_train.shape, y_train.shape)

# Reshaping, rescaling and one-hot encoding
df_train = np.reshape(df_train, (-1, 3, 32, 32))
df_train = np.transpose(np.array(df_train), (0, 2, 3, 1))
df_train = df_train / 255
print(df_train.shape)

# Importing the Test Dataset
print("For Test Dataset:")
df_test = pd.read_csv("../input/cifar10/test_x.csv")
y_test = pd.read_csv("../input/cifar10/test_y.csv")
df_test = np.array(df_test)
y_test = np.array(y_test)
print(df_test.shape, y_test.shape)

# Reshaping the dataset
df_test = np.reshape(df_test, (-1, 3, 32, 32))
print(df_test.shape)

# Reshaping, rescaling and one-hot encoding
df_test = np.transpose(np.array(df_test), (0, 2, 3, 1))
df_test = df_test / 255
y_test_oh = tf.one_hot(np.ravel(y_test), depth = 10)
print(df_test.shape, y_test_oh.shape)

For Train Dataset:
(40006, 3072) (40006, 1)
(40006, 32, 32, 3)
For Test Dataset:
(10000, 3072) (10000, 1)
(10000, 3, 32, 32)
(10000, 32, 32, 3) (10000, 10)


# 3. Performing the Augmentations on the Training Set
## 3.1. GAN-Based Augmentation

In [5]:
df_gan_aug = pd.read_csv("../input/cifar10/df_clsper_gan_aug.csv")
y_gan_aug = pd.read_csv("../input/cifar10/y_clsper_gan_aug.csv")
df_gan_aug = np.array(df_gan_aug)
y_gan_aug = np.array(y_gan_aug)

# Reshaping, rescaling and one-hot encoding
df_gan_aug = np.reshape(df_gan_aug, (-1, 3, 32, 32))
df_gan_aug = np.transpose(np.array(df_gan_aug), (0, 2, 3, 1))
print(df_gan_aug.shape, y_gan_aug.shape)

(10048, 32, 32, 3) (10048, 1)


## 3.2. Traditional Augmentation

In [6]:
df_trad_aug = pd.read_csv("../input/cifar10/df_clsper_trad_aug.csv")
y_trad_aug = pd.read_csv("../input/cifar10/y_clsper_trad_aug.csv")
df_trad_aug = np.array(df_trad_aug)
y_trad_aug = np.array(y_trad_aug)

# Reshaping, rescaling and one-hot encoding
df_trad_aug = np.reshape(df_trad_aug, (-1, 32, 32, 3))
print(df_trad_aug.shape, y_trad_aug.shape)

(10074, 32, 32, 3) (10074, 1)


## 3.3. Preparing the Augmented Training Set

In [7]:
# Concatenating the Training with Augmenting Dataset
df_aug = np.concatenate([df_train, df_gan_aug, df_trad_aug], axis=0)
y_aug = np.concatenate([y_train, y_gan_aug, y_trad_aug], axis=0)

# Creating a random permutation & shuffling the dataset
perm = np.random.permutation(df_aug.shape[0])
df_aug = np.array(df_aug[perm, : , : , : ])
y_aug = y_aug[perm]

# One-Hot Encoding
y_aug_oh = tf.one_hot(np.ravel(y_aug), depth = 10)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

(60128, 32, 32, 3) (60128, 1) (60128, 10)


# 4. Training the Model
## 4.1. Preparing the Baseline Model and the Augmented Training Set

In [8]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

# Creating Batches from the Augmented Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((df_aug, y_aug_oh)).batch(32)

In [9]:
# If the model has been pre-trained
try:
    conv_model = cnn_model((32, 32, 3))
    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    conv_model.load_weights("../input/dcai-rw/hybrid_trad_gan_augmented.h5")

# If the model hasn't been pre-trained
except:
    num_epochs = [10, 20, 30, 40, 50]
    train_loss, test_loss, train_acc, test_acc = [], [], [], []

    for epochs in num_epochs:
        # Training the Model
        conv_model = cnn_model((32, 32, 3))
        conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
        conv_model.fit(train_dataset, epochs = epochs)

        # Predicting on the Train/Test Datasets
        preds_train = conv_model.predict(df_aug)
        preds_test = conv_model.predict(df_test)

        # Finding the Predicted Classes
        cls_train = np.argmax(preds_train, axis = 1)
        cls_test = np.argmax(preds_test, axis = 1)

        # Finding the Train/Test set Loss
        train_loss.append(log_loss(y_aug_oh, preds_train))
        test_loss.append(log_loss(y_test_oh, preds_test))
        train_acc.append(accuracy_score(y_aug, cls_train))
        test_acc.append(accuracy_score(y_test, cls_test))

        print("For ", epochs, " Epochs:")
        print("Log-loss for Train Dataset = ", train_loss[-1])
        print("Log-loss for Test Dataset = ", test_loss[-1])
        print("Accuracy for Train Dataset = ", train_acc[-1])
        print("Accuracy for Test Dataset = ", test_acc[-1])
        print()
        
    # Training the Model with the best hyper-parameter settings
    ind = np.argmax(test_acc)
    best_num_epochs = num_epochs[ind]
    conv_model = cnn_model((32, 32, 3))
    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    conv_model.fit(train_dataset, epochs = best_num_epochs)

    # Saving the model along with it's weights
    conv_model.save('hybrid_trad_gan_augmented.h5')

Epoch 1/10
1879/1879 [==============================] - 15s 4ms/step - loss: 1.6392 - accuracy: 0.3882
Epoch 2/10
1879/1879 [==============================] - 8s 4ms/step - loss: 1.2444 - accuracy: 0.5511
Epoch 3/10
1879/1879 [==============================] - 9s 5ms/step - loss: 1.0715 - accuracy: 0.6239
Epoch 4/10
1879/1879 [==============================] - 9s 5ms/step - loss: 0.9653 - accuracy: 0.6605
Epoch 5/10
1879/1879 [==============================] - 8s 4ms/step - loss: 0.8932 - accuracy: 0.6866
Epoch 6/10
1879/1879 [==============================] - 9s 5ms/step - loss: 0.8415 - accuracy: 0.7054
Epoch 7/10
1879/1879 [==============================] - 8s 4ms/step - loss: 0.7988 - accuracy: 0.7194
Epoch 8/10
1879/1879 [==============================] - 8s 4ms/step - loss: 0.7586 - accuracy: 0.7344
Epoch 9/10
1879/1879 [==============================] - 9s 5ms/step - loss: 0.7294 - accuracy: 0.7448
Epoch 10/10
1879/1879 [==============================] - 8s 4ms/step - loss: 0.69

## 4.2. Predicting the Performance

In [10]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_aug)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Augmented Dataset = ", log_loss(y_aug_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Weighted F1 Score for Augmented Dataset = ", f1_score(y_aug, cls_train, average = 'weighted'))
print("Weighted F1 Score for Test Dataset = ", f1_score(y_test, cls_test, average = 'weighted'))
print("Accuracy for Augmented Dataset = ", accuracy_score(y_aug, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

Log-loss for Augmented Dataset =  0.17044280393337208
Log-loss for Test Dataset =  0.7709342780244012
Weighted F1 Score for Augmented Dataset =  0.9476430446272331
Weighted F1 Score for Test Dataset =  0.76664537228173
Accuracy for Augmented Dataset =  0.9477780734433209
Accuracy for Test Dataset =  0.7698
